# Data Preparation


In this notebook will contain all the process regarding data preparation. Starting transforming the data to pandas dataframes and then profiling the data to have a preview of the problem domain. Then I will start doing some correlations among the datasets and visualize some characteristics before proceeding with the feature engineering. In adition, we will add some moving averages and other trading technical indicators. At the end, we might consider a PCA analysis to reduce the features for better analysis. 

## Import Data and Transforme to Consistent Pandas Data Frames

In [1]:
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from collections import namedtuple
from IPython.display import display, HTML
import functools
%matplotlib inline
# Optional for interactive
# %matplotlib notebook (watch video for full details)

In [2]:
# Accounting format for floats for pandas.

pd.options.display.float_format = '{:,.2f}'.format

The raw files are in the folder raw_data. We have a total of 8 files for the exchanges buda(Peru, Colombia and Chile) and mercadobitcoin (Brazil). In the following cell I am going to extract the data to pandas dataframes

In [3]:
def dateparse_buda (time_in_secs): 
    return datetime.fromtimestamp(float(time_in_secs)/1000)

In [4]:
df_btc_clp=pd.read_csv('raw_data/buda_btc-clp_20161101_20180312.csv',index_col='datetime',parse_dates=True, date_parser=dateparse_buda)
df_btc_cop=pd.read_csv('raw_data/buda_btc-cop_20161101_20180312.csv',index_col='datetime',parse_dates=True,date_parser=dateparse_buda)
df_btc_pen=pd.read_csv('raw_data/buda_btc-pen_20161101_20180312.csv',index_col='datetime',parse_dates=True,date_parser=dateparse_buda)
df_eth_btc=pd.read_csv('raw_data/buda_eth-btc_20170701_20180312.csv',index_col='datetime',parse_dates=True,date_parser=dateparse_buda)
df_eth_clp=pd.read_csv('raw_data/buda_eth-clp_20170501_20180312.csv',index_col='datetime',parse_dates=True,date_parser=dateparse_buda)
df_eth_cop=pd.read_csv('raw_data/buda_eth-cop_20170601_20180312.csv',index_col='datetime',parse_dates=True,date_parser=dateparse_buda)
df_eth_pen=pd.read_csv('raw_data/buda_eth-pen_20170601_20180312.csv',index_col='datetime',parse_dates=True,date_parser=dateparse_buda)
df_btc_brl=pd.read_csv('raw_data/mercadobitcoin_BTC_20130612_20180312.csv',index_col='date',parse_dates=True)
Dataset=namedtuple('Dataset','exchange df')

datasets={'btc_clp':Dataset('Buda',df_btc_clp.sort_index()),
          'btc_cop':Dataset('Buda',df_btc_cop.sort_index()),
          'btc_pen':Dataset('Buda',df_btc_pen.sort_index()),
          'eth_btc':Dataset('Buda',df_eth_btc.sort_index()),
          'eth_clp':Dataset('Buda',df_eth_clp.sort_index()),
          'eth_cop':Dataset('Buda',df_eth_cop.sort_index()),
          'eth_pen':Dataset('Buda',df_eth_pen.sort_index()),
          'btc_brl':Dataset('MercadoBitcoin',df_btc_brl.sort_index())}
datasets['btc_brl'].df.head(10)


,lowest,volume,amount,avg_price,opening,closing,highest,quantity
date,,,,,,,,
2013-06-12,249.00,"2,799.69",11,256.45,249.00,265.00,275.00,10.92
2013-06-13,259.00,"2,830.41",16,266.40,265.00,269.00,269.00,10.62
2013-06-14,245.00,"8,694.71",35,255.42,267.00,250.00,268.00,34.04
2013-06-15,246.01,"4,481.41",8,256.87,250.00,246.01,259.99,17.45
2013-06-16,246.01,427.69,14,256.22,246.01,252.00,257.43,1.67
2013-06-17,252.00,"3,628.96",12,254.52,257.43,254.96,257.43,14.26
2013-06-18,246.11,"7,498.59",22,256.79,254.95,263.00,263.00,29.20
2013-06-19,251.01,"1,137.09",18,262.96,251.01,260.00,264.98,4.32
2013-06-20,260.00,"7,253.13",28,267.51,263.00,269.00,269.00,27.11


In [5]:
def all_datasets(func,datasets ):
    for key,dataset in datasets.items():
        display(HTML('<h3>'+key+'</h3>'))
        print(func.__name__)
        datasets[key]=func(dataset)

def copy_all_datasets(datasets):
    new_dataset={}
    for key,dataset in datasets.items():
        new_dataset[key]=Dataset(dataset.exchange,dataset.df.copy())
    return  new_dataset       

In [6]:
def rename_mercado_bitcoin(dataset):
    
    if dataset.exchange=='MercadoBitcoin': 
        columns_standard={'opening':'open'
                          , 'closing':'close'
                          , 'highest':'high'
                          , 'lowest':'low'}
        dataset.df.rename(columns=columns_standard,inplace=True)
        dataset.df.index.rename('datetime',inplace=True)
    
    return dataset

   
        
all_datasets(rename_mercado_bitcoin,datasets)

rename_mercado_bitcoin


rename_mercado_bitcoin


rename_mercado_bitcoin


rename_mercado_bitcoin


rename_mercado_bitcoin


rename_mercado_bitcoin


rename_mercado_bitcoin


rename_mercado_bitcoin


For the exchange buda, there are multiple entries for each day. Therefore it is necesary to resample the data for each day to have a consistant information vs mercado bitcoin.

In [7]:
df_btc_cop.resample(rule='D').count().tail(50)
df_btc_cop['2018-03-11']

,open,high,low,close,volume
datetime,,,,,
2018-03-11 05:00:00,"24,300,700.00","25,499,990.00","24,300,700.00","24,300,707.00",0.36
2018-03-11 13:00:00,"25,499,987.00","25,796,997.00","24,900,001.00","25,796,997.00",0.21
2018-03-11 21:00:00,"25,400,000.00","25,400,000.00","25,400,000.00","25,400,000.00",0.05


In [8]:
def first_entry(entry):
    if entry.size==0:
        return None 
    return entry[0]
def last_entry(entry):
    if entry.size==0:
        return None 
    return entry[-1]


In [9]:
def fix_daily(dataset):
    if dataset.exchange=='Buda':
        open_series=dataset.df['open'].resample(rule='D').apply(first_entry)
        close_series=dataset.df['close'].resample(rule='D').apply(last_entry)
        high_series=dataset.df['high'].resample(rule='D').max()
        low_series=dataset.df['low'].resample(rule='D').min()
        volume_series=dataset.df['volume'].resample(rule='D').sum()
        dataframe_daily=pd.DataFrame(open_series)
        dataframe_daily=dataframe_daily.merge(pd.DataFrame(close_series),left_index=True,right_index=True)
        dataframe_daily=dataframe_daily.merge(pd.DataFrame(high_series),left_index=True,right_index=True)
        dataframe_daily=dataframe_daily.merge(pd.DataFrame(low_series),left_index=True,right_index=True)
        dataframe_daily=dataframe_daily.merge(pd.DataFrame(volume_series),left_index=True,right_index=True)
        dataset=Dataset('Buda',dataframe_daily )
    return dataset



In [10]:
all_datasets(fix_daily,datasets)



fix_daily


fix_daily


fix_daily


fix_daily


fix_daily


fix_daily


fix_daily


fix_daily


### Fill Missing Values

For some days there are missing values therefore we are going to apply the forward fill method of pandas to fill the closing price. The open, low and high will be assigned to the close value of the next prevoius day with information available. For the volume it will be set as 0.

In [11]:

datasets['btc_cop'].df.head(10)

,open,close,high,low,volume
datetime,,,,,
2016-10-31,"2,099,950.00","2,100,000.00","2,100,000.00","2,099,950.00",2.28
2016-11-01,"2,115,664.74","2,100,541.00","2,115,664.74","2,100,541.00",0.54
2016-11-02,"2,200,000.00","2,200,000.00","2,200,000.00","2,200,000.00",0.86
2016-11-03,"2,250,000.00","2,038,008.07","2,250,000.00","2,026,909.28",4.66
2016-11-04,"2,173,361.50","2,192,224.31","2,192,224.31","2,001,026.82",2.76
2016-11-05,nan,nan,nan,nan,nan
2016-11-06,nan,nan,nan,nan,nan
2016-11-07,nan,nan,nan,nan,nan
2016-11-08,"2,040,100.23","2,030,280.91","2,041,656.90","2,021,216.07",3.52


In [12]:
def fill_nulls(dataset):
    dataset.df['close'].fillna(method='ffill', inplace=True)
    dataset.df['close'].fillna(method='bfill', inplace=True)
    dataset.df['open'].fillna(dataset.df['close'], inplace=True)
    dataset.df['high'].fillna(dataset.df['close'], inplace=True)
    dataset.df['low'].fillna(dataset.df['close'], inplace=True)
    dataset.df['volume'].fillna(0, inplace=True)
    return dataset

all_datasets(fill_nulls,datasets)

datasets['btc_cop'].df.head(10)

fill_nulls


fill_nulls


fill_nulls


fill_nulls


fill_nulls


fill_nulls


fill_nulls


fill_nulls


,open,close,high,low,volume
datetime,,,,,
2016-10-31,"2,099,950.00","2,100,000.00","2,100,000.00","2,099,950.00",2.28
2016-11-01,"2,115,664.74","2,100,541.00","2,115,664.74","2,100,541.00",0.54
2016-11-02,"2,200,000.00","2,200,000.00","2,200,000.00","2,200,000.00",0.86
2016-11-03,"2,250,000.00","2,038,008.07","2,250,000.00","2,026,909.28",4.66
2016-11-04,"2,173,361.50","2,192,224.31","2,192,224.31","2,001,026.82",2.76
2016-11-05,"2,192,224.31","2,192,224.31","2,192,224.31","2,192,224.31",0.00
2016-11-06,"2,192,224.31","2,192,224.31","2,192,224.31","2,192,224.31",0.00
2016-11-07,"2,192,224.31","2,192,224.31","2,192,224.31","2,192,224.31",0.00
2016-11-08,"2,040,100.23","2,030,280.91","2,041,656.90","2,021,216.07",3.52


### Profiling Data

Describe each dataset of the raw data.

In [13]:
#Index(['lowest', 'volume', 'amount', 'avg_price', 'opening', 'closing',
#       'highest', 'quantity'],
#      dtype='object')
def describe_dataset( dataset):
    display(HTML('<H1>Range Dates</H1>'))
    display(HTML('<H3>Min: '+dataset.df.index.min().strftime('%Y-%m-%d')+'</H3>'))
    display(HTML('<H3>Min: '+dataset.df.index.max().strftime('%Y-%m-%d')+'</H3>'))
    display(HTML('<H1>Min</H1>'))
    print(dataset.df.idxmin())
    display(HTML('<H1>Max</H1>'))
    print(dataset.df.idxmax())    
    display(HTML(dataset.df.describe().to_html()))
    return  dataset 
all_datasets(describe_dataset,datasets)

describe_dataset


open     2016-11-04
close    2016-11-12
high     2016-11-07
low      2016-11-03
volume   2016-12-31
dtype: datetime64[ns]


open     2017-12-18
close    2017-12-17
high     2017-12-07
low      2017-12-17
volume   2017-03-10
dtype: datetime64[ns]


,open,close,high,low,volume
count,497.00,497.00,497.00,497.00,497.00
mean,"3,031,204.69","3,042,093.59","3,185,056.81","2,859,642.35",34.90
std,"3,012,846.52","3,020,119.25","3,199,563.20","2,793,238.47",29.00
min,"465,164.77","463,059.45","472,933.11","449,879.51",0.00
25%,"740,000.00","740,000.00","774,500.00","717,653.89",15.81
50%,"1,799,000.00","1,797,000.00","1,860,000.00","1,726,270.00",29.20
75%,"4,549,808.11","4,601,423.13","4,875,000.00","4,280,101.00",43.27
max,"13,794,500.00","13,794,997.00","14,200,000.00","13,000,000.00",233.56


describe_dataset


...

,open,close,high,low,volume
count,208.00,208.00,208.00,208.00,208.00
mean,"1,956.52","1,994.46","2,219.63","1,853.97",3.08
std,"1,062.31","1,105.04","1,391.74",996.39,5.00
min,300.00,654.01,830.00,1.01,0.00
25%,"1,028.31","1,047.88","1,050.75","1,017.65",0.10
50%,"1,463.06","1,447.00","1,564.50","1,308.92",0.93
75%,"2,779.73","2,858.75","3,104.41","2,717.64",3.98
max,"4,994.00","4,997.00","8,000.00","4,994.00",37.77


describe_dataset


low         2018-02-16
volume      2013-08-18
amount      2013-08-18
avg_price   2013-07-09
open        2013-07-04
close       2013-07-03
high        2013-07-10
quantity    2013-08-18
dtype: datetime64[ns]


low         2017-12-18
volume      2017-12-07
amount      2017-12-07
avg_price   2017-12-18
open        2017-12-19
close       2017-12-18
high        2017-12-17
quantity    2017-11-29
dtype: datetime64[ns]


,low,volume,amount,avg_price,open,close,high,quantity
count,"1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00"
mean,"5,167.44","2,147,448.68","1,290.21","5,432.23","5,421.96","5,437.67","5,641.17",158.60
std,"10,082.28","7,653,198.97","2,837.94","10,674.96","10,672.42","10,676.18","11,164.10",196.91
min,0.00,24.55,2.00,200.24,193.00,190.01,209.00,0.10
25%,936.14,"59,930.05",161.00,962.42,959.70,960.00,985.00,52.42
50%,"1,630.05","131,661.52",306.00,"1,690.37","1,695.83","1,690.11","1,729.50",102.75
75%,"2,881.00","487,363.80",839.00,"2,945.76","2,949.58","2,960.00","3,046.00",181.59
max,"67,500.00","108,023,418.47","33,169.00","68,942.87","68,800.00","69,099.00","69,950.00","2,629.60"


### Adding Technical Indicators on the Closing Price

Add technical Indicators on the closing prices. Bollinger Bands®, Exponential Moving Average (EMA), Moving Average Converge Divergence (MACD), Heikin-Ashi and Daily Returns.

In [14]:
def adding_technical_indicators(dataset):
    dataset.df['EWMA26']=dataset.df['close'].ewm(span=26).mean()
    dataset.df['EWMA12']=dataset.df['close'].ewm(span=12).mean()
    dataset.df['EWMA9']=dataset.df['close'].ewm(span=9).mean()
    dataset.df['MACD']=dataset.df['EWMA26']-dataset.df['EWMA12']
    dataset.df['Returns']=dataset.df['close'].pct_change(1)
    dataset.df['log_return']=np.log(dataset.df['close']/dataset.df['close'].shift(1))
    dataset.df['Bollinger Upper']=dataset.df['EWMA12']+2*dataset.df['close'].ewm(span=12).std()
    dataset.df['Bollinger Lower']=dataset.df['EWMA12']-2*dataset.df['close'].ewm(span=12).std()
    dataset.df['Heiking_Close']=(dataset.df['close']+dataset.df['high']+dataset.df['low']+dataset.df['open'])/4
    dataset.df['Heiking High']=dataset.df[['close','high','open']].max(axis=1)
    dataset.df['Heiking Low']=dataset.df[['close','low','open']].min(axis=1)
    dataset.df['Heiking Open']=(dataset.df['close'].shift(1)+dataset.df['open'].shift(1))/2
    dataset.df['log_MACD']=np.log(dataset.df['EWMA26']/dataset.df['EWMA12'])
    dataset.df['Variance12']=dataset.df['close'].ewm(span=12).std()
    return dataset
 
all_datasets(adding_technical_indicators,datasets)
# datasets['btc_cop'].df.head(10)


    
    
    

adding_technical_indicators


adding_technical_indicators


adding_technical_indicators


adding_technical_indicators


adding_technical_indicators


adding_technical_indicators


adding_technical_indicators


adding_technical_indicators


In [15]:
all_datasets(describe_dataset,datasets)

describe_dataset


open              2016-11-04
close             2016-11-12
high              2016-11-07
low               2016-11-03
volume            2016-12-31
EWMA26            2016-11-07
EWMA12            2016-11-07
EWMA9             2016-11-07
MACD              2017-12-17
Returns           2017-12-21
log_return        2017-12-21
Bollinger Upper   2016-11-07
Bollinger Lower   2016-11-07
Heiking_Close     2016-11-04
Heiking High      2016-11-07
Heiking Low       2016-11-03
Heiking Open      2016-11-08
log_MACD          2017-12-11
Variance12        2016-11-02
dtype: datetime64[ns]


open              2017-12-18
close             2017-12-17
high              2017-12-07
low               2017-12-17
volume            2017-03-10
EWMA26            2017-12-26
EWMA12            2017-12-20
EWMA9             2017-12-19
MACD              2018-02-05
Returns           2018-02-06
log_return        2018-02-06
Bollinger Upper   2017-12-18
Bollinger Lower   2018-01-07
Heiking_Close     2017-12-17
Heiking High      2017-12-07
Heiking Low       2017-12-17
Heiking Open      2017-12-18
log_MACD          2018-02-05
Variance12        2017-12-11
dtype: datetime64[ns]


,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,497.00,497.00,497.00,497.00,497.00,497.00,497.00,497.00,497.00,496.00,496.00,496.00,496.00,497.00,497.00,497.00,496.00,497.00,496.00
mean,"3,031,204.69","3,042,093.59","3,185,056.81","2,859,642.35",34.90,"2,898,503.91","2,980,340.39","2,997,697.75","-81,836.48",0.01,0.01,"3,641,658.75","2,329,065.37","3,029,499.36","3,185,056.81","2,859,642.35","3,031,251.27",-0.03,"328,148.34"
std,"3,012,846.52","3,020,119.25","3,199,563.20","2,793,238.47",29.00,"2,879,587.88","2,964,858.01","2,981,518.29","386,711.81",0.06,0.06,"3,823,001.18","2,168,641.98","3,000,240.25","3,199,563.20","2,793,238.47","3,013,147.92",0.06,"464,647.70"
min,"465,164.77","463,059.45","472,933.11","449,879.51",0.00,"473,295.93","471,806.01","470,974.61","-1,921,099.58",-0.25,-0.29,"490,404.94","453,207.08","466,176.85","472,933.11","449,879.51","464,631.64",-0.19,"3,706.92"
25%,"740,000.00","740,000.00","774,500.00","717,653.89",15.81,"739,805.16","725,905.40","723,704.00","-155,751.41",-0.02,-0.02,"830,771.70","627,777.71","750,409.62","774,500.00","717,653.89","743,529.38",-0.07,"43,783.90"
50%,"1,799,000.00","1,797,000.00","1,860,000.00","1,726,270.00",29.20,"1,737,612.80","1,747,776.04","1,744,424.72","-21,359.65",0.00,0.00,"2,034,059.33","1,548,350.56","1,790,333.75","1,860,000.00","1,726,270.00","1,789,838.00",-0.02,"152,863.73"
75%,"4,549,808.11","4,601,423.13","4,875,000.00","4,280,101.00",43.27,"4,118,139.14","4,484,995.18","4,541,210.86","1,924.95",0.03,0.03,"5,252,634.25","3,342,575.74","4,529,125.50","4,875,000.00","4,280,101.00","4,528,678.29",0.00,"367,989.29"
max,"13,794,500.00","13,794,997.00","14,200,000.00","13,000,000.00",233.56,"10,385,822.57","12,058,383.38","12,414,776.40","1,087,897.67",0.37,0.32,"16,150,609.07","8,447,717.71","13,573,749.25","14,200,000.00","13,000,000.00","13,647,498.50",0.17,"2,267,863.23"


describe_dataset


...

,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,208.00,208.00,208.00,208.00,208.00,208.00,208.00,208.00,208.00,207.00,207.00,207.00,207.00,208.00,208.00,208.00,207.00,208.00,207.00
mean,"1,956.52","1,994.46","2,219.63","1,853.97",3.08,"1,890.21","1,952.20","1,964.54",-61.99,0.03,0.00,"2,677.95","1,235.67","2,006.14","2,219.63","1,853.97","1,973.86",-0.03,360.57
std,"1,062.31","1,105.04","1,391.74",996.39,5.00,956.56,"1,001.26","1,011.36",159.84,0.25,0.22,"1,537.97",781.53,"1,103.38","1,391.74",996.39,"1,067.08",0.06,348.26
min,300.00,654.01,830.00,1.01,0.00,969.41,916.63,893.87,-689.58,-0.57,-0.84,"1,001.07",-137.57,660.20,830.00,1.01,734.95,-0.25,0.79
25%,"1,028.31","1,047.88","1,050.75","1,017.65",0.10,"1,041.15","1,031.29","1,030.12",-120.54,-0.05,-0.05,"1,282.14",726.43,"1,045.58","1,050.75","1,017.65","1,045.54",-0.05,126.15
50%,"1,463.06","1,447.00","1,564.50","1,308.92",0.93,"1,313.14","1,433.05","1,435.34",-6.16,0.00,0.00,"2,159.86",908.15,"1,438.38","1,564.50","1,308.92","1,444.00",-0.00,270.47
75%,"2,779.73","2,858.75","3,104.41","2,717.64",3.98,"2,994.16","2,985.70","2,958.08",26.48,0.06,0.06,"4,116.32","1,981.05","2,896.43","3,104.41","2,717.64","2,872.01",0.02,438.58
max,"4,994.00","4,997.00","8,000.00","4,994.00",37.77,"3,457.97","3,781.11","3,919.97",182.05,1.75,1.01,"6,453.56","2,954.51","4,994.00","8,000.00","4,994.00","4,994.00",0.06,"1,555.51"


describe_dataset


low               2018-02-16
volume            2013-08-18
amount            2013-08-18
avg_price         2013-07-09
open              2013-07-04
close             2013-07-03
high              2013-07-10
quantity          2013-08-18
EWMA26            2013-07-28
EWMA12            2013-07-11
EWMA9             2013-07-10
MACD              2017-12-18
Returns           2013-09-06
log_return        2013-09-06
Bollinger Upper   2013-07-28
Bollinger Lower   2013-07-09
Heiking_Close     2013-07-07
Heiking High      2013-07-10
Heiking Low       2018-02-16
Heiking Open      2013-07-08
log_MACD          2013-11-29
Variance12        2013-06-13
dtype: datetime64[ns]


low               2017-12-18
volume            2017-12-07
amount            2017-12-07
avg_price         2017-12-18
open              2017-12-19
close             2017-12-18
high              2017-12-17
quantity          2017-11-29
EWMA26            2018-01-08
EWMA12            2017-12-20
EWMA9             2017-12-20
MACD              2018-02-06
Returns           2013-11-18
log_return        2013-11-18
Bollinger Upper   2017-12-18
Bollinger Lower   2018-01-08
Heiking_Close     2017-12-18
Heiking High      2017-12-17
Heiking Low       2017-12-18
Heiking Open      2017-12-19
log_MACD          2018-02-06
Variance12        2017-12-18
dtype: datetime64[ns]


,low,volume,amount,avg_price,open,close,high,quantity,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,"1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,680.00","1,680.00","1,680.00","1,680.00","1,681.00","1,681.00","1,681.00","1,680.00","1,681.00","1,680.00"
mean,"5,167.44","2,147,448.68","1,290.21","5,432.23","5,421.96","5,437.67","5,641.17",158.60,"5,190.20","5,331.35","5,361.39",-141.15,0.00,0.00,"6,424.19","4,244.54","5,417.06","5,641.17","5,167.44","5,414.56",-0.01,544.91
std,"10,082.28","7,653,198.97","2,837.94","10,674.96","10,672.42","10,676.18","11,164.10",196.91,"10,150.19","10,463.81","10,526.73","1,058.12",0.05,0.05,"13,160.45","7,863.24","10,633.05","11,164.10","10,082.28","10,653.03",0.06,"1,411.80"
min,0.00,24.55,2.00,200.24,193.00,190.01,209.00,0.10,223.00,215.88,211.36,"-9,240.80",-0.24,-0.28,235.46,162.04,201.14,209.00,0.00,199.28,-0.29,2.83
25%,936.14,"59,930.05",161.00,962.42,959.70,960.00,985.00,52.42,954.78,947.68,946.06,-66.23,-0.01,-0.01,"1,035.60",852.43,960.68,985.00,936.14,958.30,-0.03,35.22
50%,"1,630.05","131,661.52",306.00,"1,690.37","1,695.83","1,690.11","1,729.50",102.75,"1,699.56","1,690.84","1,690.69",-8.19,0.00,0.00,"1,801.39","1,440.25","1,682.00","1,729.50","1,630.05","1,685.40",-0.01,70.72
75%,"2,881.00","487,363.80",839.00,"2,945.76","2,949.58","2,960.00","3,046.00",181.59,"2,997.09","2,961.60","2,956.66",21.45,0.02,0.02,"3,293.18","2,589.79","2,950.69","3,046.00","2,881.00","2,953.66",0.02,208.76
max,"67,500.00","108,023,418.47","33,169.00","68,942.87","68,800.00","69,099.00","69,950.00","2,629.60","52,276.97","60,036.26","61,938.78","5,368.79",0.41,0.35,"80,143.92","45,960.29","68,599.50","69,950.00","67,500.00","68,499.50",0.17,"10,544.25"


## Rescaling the data

We will rescale the data to the maximum closing price. The date will be 2017-12-18  witch is the date where the bitcoin has the global maximum value along the dataset. 



In [16]:
rescaled_datasets=copy_all_datasets(datasets)


In [17]:
class Rescaler:
    def __init__(self, date_to_scale):
        self.date_to_scale=date_to_scale
        
    def rescale(self,dataset):
        closing_price_at_max=dataset.df.loc[self.date_to_scale]['close']
        column_exceptions=['volume','Returns','quantity']
        valid_colummns=[valid_column for valid_column in dataset.df.columns if valid_column not in column_exceptions]
        for colummn in valid_colummns:
            dataset.df[colummn]=dataset.df[colummn]/closing_price_at_max
        return dataset

max_date= datetime(2017,12,18)
rescaler=Rescaler(max_date)
all_datasets(rescaler.rescale,rescaled_datasets)

rescale


rescale


rescale


rescale


rescale


rescale


rescale


rescale


## Apply log to all fields 


Log transforms are useful when applied to skewed distributions as they tend to expand the values which fall in the range of lower magnitudes and tend to compress or reduce the values which fall in the range of higher ([Feature Engineering](https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b)). 


In [18]:
log_datasets=copy_all_datasets(datasets) 
log_divided_close_datasets=copy_all_datasets(datasets) 

def log_transform(dataset):
    column_exceptions=['log_return', 'Returns','volume','log_MACD']
    valid_colummns=[valid_column for valid_column in dataset.df.columns if valid_column not in column_exceptions]
    for colummn in valid_colummns:
        dataset.df[colummn]=np.log(dataset.df[colummn])
    return dataset

def log_transform_divided_by_close(dataset):
    column_exceptions=['log_return', 'Returns','volume','log_MACD','close','Variance12']
    valid_colummns=[valid_column for valid_column in dataset.df.columns if valid_column not in column_exceptions]
    for colummn in valid_colummns:
        dataset.df[colummn]=np.log(dataset.df[colummn]/dataset.df['close'])
    return dataset


all_datasets(log_transform,log_datasets)
all_datasets(log_transform_divided_by_close,log_divided_close_datasets)


log_transform


/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  
/Users/JR/anaconda3/envs/pyfinance/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


log_transform


log_transform


log_transform


...

log_transform_divided_by_close


log_transform_divided_by_close


log_transform_divided_by_close


log_transform_divided_by_close


log_transform_divided_by_close


In [19]:
log_divided_close_datasets['btc_clp'].df.tail()

,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
datetime,,,,,,,,,,,,,,,,,,,
2018-03-07,0.08,"6,000,011.00",0.11,-0.07,76.05,0.06,0.07,0.07,nan,-0.06,-0.06,0.18,-0.06,0.03,0.11,-0.07,0.09,-0.00,"371,407.47"
2018-03-08,0.11,"5,479,560.00",0.14,-0.01,42.10,0.14,0.13,0.13,-4.79,-0.09,-0.09,0.28,-0.04,0.06,0.14,-0.01,0.13,0.01,"488,927.05"
2018-03-09,-0.07,"5,792,598.00",0.03,-0.13,28.90,0.08,0.07,0.06,-4.29,0.06,0.06,0.21,-0.10,-0.04,0.03,-0.13,-0.00,0.01,"483,683.25"
2018-03-10,0.05,"5,498,000.00",0.07,-0.03,21.64,0.12,0.10,0.09,-3.75,-0.05,-0.05,0.26,-0.08,0.02,0.07,-0.03,0.02,0.02,"516,139.53"
2018-03-11,-0.08,"5,929,988.95",0.01,-0.08,9.94,0.04,0.02,0.01,-3.80,0.08,0.08,0.17,-0.15,-0.04,0.01,-0.08,-0.05,0.02,"478,378.89"


In [20]:
all_datasets(describe_dataset,log_datasets)

describe_dataset


open              2016-11-04
close             2016-11-12
high              2016-11-07
low               2016-11-03
volume            2016-12-31
EWMA26            2016-11-07
EWMA12            2016-11-07
EWMA9             2016-11-07
MACD              2016-11-14
Returns           2017-12-21
log_return        2017-12-21
Bollinger Upper   2016-11-07
Bollinger Lower   2016-11-07
Heiking_Close     2016-11-04
Heiking High      2016-11-07
Heiking Low       2016-11-03
Heiking Open      2016-11-08
log_MACD          2017-12-11
Variance12        2016-11-02
dtype: datetime64[ns]


open              2017-12-18
close             2017-12-17
high              2017-12-07
low               2017-12-17
volume            2017-03-10
EWMA26            2017-12-26
EWMA12            2017-12-20
EWMA9             2017-12-19
MACD              2018-02-05
Returns           2018-02-06
log_return        2018-02-06
Bollinger Upper   2017-12-18
Bollinger Lower   2018-01-07
Heiking_Close     2017-12-17
Heiking High      2017-12-07
Heiking Low       2017-12-17
Heiking Open      2017-12-18
log_MACD          2018-02-05
Variance12        2017-12-11
dtype: datetime64[ns]


,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,497.00,497.00,497.00,497.00,497.00,497.00,497.00,497.00,149.00,496.00,496.00,496.00,496.00,497.00,497.00,497.00,496.00,497.00,496.00
mean,14.45,14.46,14.49,14.41,34.90,14.40,14.43,14.44,-inf,0.01,0.01,14.59,14.25,14.45,14.49,14.41,14.45,-0.03,11.78
std,0.98,0.99,0.99,0.97,29.00,0.98,0.99,0.99,nan,0.06,0.06,1.04,0.92,0.98,0.99,0.97,0.98,0.06,1.46
min,13.05,13.05,13.07,13.02,0.00,13.07,13.06,13.06,-inf,-0.25,-0.29,13.10,13.02,13.05,13.07,13.02,13.05,-0.19,8.22
25%,13.51,13.51,13.56,13.48,15.81,13.51,13.50,13.49,8.69,-0.02,-0.02,13.63,13.35,13.53,13.56,13.48,13.52,-0.07,10.69
50%,14.40,14.40,14.44,14.36,29.20,14.37,14.37,14.37,10.72,0.00,0.00,14.53,14.25,14.40,14.44,14.36,14.40,-0.02,11.94
75%,15.33,15.34,15.40,15.27,43.27,15.23,15.32,15.33,12.19,0.03,0.03,15.47,15.02,15.33,15.40,15.27,15.33,0.00,12.82
max,16.44,16.44,16.47,16.38,233.56,16.16,16.31,16.33,13.90,0.37,0.32,16.60,15.95,16.42,16.47,16.38,16.43,0.17,14.63


describe_dataset


...

,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,208.00,208.00,208.00,208.00,208.00,208.00,208.00,208.00,95.00,207.00,207.00,207.00,204.00,208.00,208.00,208.00,207.00,208.00,207.00
mean,7.43,7.45,7.52,7.36,3.08,7.42,7.44,7.45,-inf,0.03,0.00,7.72,6.93,7.45,7.52,7.36,7.44,-0.03,5.43
std,0.55,0.54,0.59,0.73,5.00,0.50,0.52,0.52,nan,0.25,0.22,0.59,0.71,0.55,0.59,0.73,0.54,0.06,1.06
min,5.70,6.48,6.72,0.01,0.00,6.88,6.82,6.80,-inf,-0.57,-0.84,6.91,3.23,6.49,6.72,0.01,6.60,-0.25,-0.23
25%,6.94,6.95,6.96,6.93,0.10,6.95,6.94,6.94,2.56,-0.05,-0.05,7.16,6.60,6.95,6.96,6.93,6.95,-0.05,4.84
50%,7.29,7.28,7.36,7.18,0.93,7.18,7.27,7.27,3.38,0.00,0.00,7.68,6.81,7.27,7.36,7.18,7.28,-0.00,5.60
75%,7.93,7.96,8.04,7.91,3.98,8.00,8.00,7.99,4.00,0.06,0.06,8.32,7.60,7.97,8.04,7.91,7.96,0.02,6.08
max,8.52,8.52,8.99,8.52,37.77,8.15,8.24,8.27,5.20,1.75,1.01,8.77,7.99,8.52,8.99,8.52,8.52,0.06,7.35


describe_dataset


low               2013-07-04
volume            2013-08-18
amount            2013-08-18
avg_price         2013-07-09
open              2013-07-04
close             2013-07-03
high              2013-07-10
quantity          2013-08-18
EWMA26            2013-07-28
EWMA12            2013-07-11
EWMA9             2013-07-10
MACD              2015-06-04
Returns           2013-09-06
log_return        2013-09-06
Bollinger Upper   2013-07-28
Bollinger Lower   2013-07-09
Heiking_Close     2013-07-07
Heiking High      2013-07-10
Heiking Low       2013-07-04
Heiking Open      2013-07-08
log_MACD          2013-11-29
Variance12        2013-06-13
dtype: datetime64[ns]


low               2017-12-18
volume            2017-12-07
amount            2017-12-07
avg_price         2017-12-18
open              2017-12-19
close             2017-12-18
high              2017-12-17
quantity          2017-11-29
EWMA26            2018-01-08
EWMA12            2017-12-20
EWMA9             2017-12-20
MACD              2018-02-06
Returns           2013-11-18
log_return        2013-11-18
Bollinger Upper   2017-12-18
Bollinger Lower   2018-01-08
Heiking_Close     2017-12-18
Heiking High      2017-12-17
Heiking Low       2017-12-18
Heiking Open      2017-12-19
log_MACD          2018-02-06
Variance12        2017-12-18
dtype: datetime64[ns]


,low,volume,amount,avg_price,open,close,high,quantity,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,"1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00",684.00,"1,680.00","1,680.00","1,680.00","1,680.00","1,681.00","1,681.00","1,681.00","1,680.00","1,681.00","1,680.00"
mean,-inf,"2,147,448.68",5.95,7.62,7.62,7.62,7.65,4.56,7.59,7.61,7.61,-inf,0.00,0.00,7.73,7.46,7.62,7.65,-inf,7.62,-0.01,4.65
std,nan,"7,653,198.97",1.50,1.24,1.24,1.24,1.24,1.05,1.22,1.23,1.23,nan,0.05,0.05,1.26,1.20,1.24,1.24,nan,1.24,0.06,1.61
min,-inf,24.55,0.69,5.30,5.26,5.25,5.34,-2.30,5.41,5.37,5.35,-inf,-0.24,-0.28,5.46,5.09,5.30,5.34,-inf,5.29,-0.29,1.04
25%,6.84,"59,930.05",5.08,6.87,6.87,6.87,6.89,3.96,6.86,6.85,6.85,2.62,-0.01,-0.01,6.94,6.75,6.87,6.89,6.84,6.87,-0.03,3.56
50%,7.40,"131,661.52",5.72,7.43,7.44,7.43,7.46,4.63,7.44,7.43,7.43,3.34,0.00,0.00,7.50,7.27,7.43,7.46,7.40,7.43,-0.01,4.26
75%,7.97,"487,363.80",6.73,7.99,7.99,7.99,8.02,5.20,8.01,7.99,7.99,4.15,0.02,0.02,8.10,7.86,7.99,8.02,7.97,7.99,0.02,5.34
max,11.12,"108,023,418.47",10.41,11.14,11.14,11.14,11.16,7.87,10.86,11.00,11.03,8.59,0.41,0.35,11.29,10.74,11.14,11.16,11.12,11.13,0.17,9.26


In [21]:
all_datasets(describe_dataset,log_divided_close_datasets)

describe_dataset


open              2018-02-06
close             2016-11-12
high              2016-10-31
low               2017-12-22
volume            2016-12-31
EWMA26            2017-12-06
EWMA12            2017-12-06
EWMA9             2017-12-06
MACD              2016-11-14
Returns           2017-12-21
log_return        2017-12-21
Bollinger Upper   2016-12-21
Bollinger Lower   2017-12-07
Heiking_Close     2018-02-06
Heiking High      2016-10-31
Heiking Low       2017-12-22
Heiking Open      2017-12-06
log_MACD          2017-12-11
Variance12        2016-11-02
dtype: datetime64[ns]


open              2017-06-14
close             2017-12-17
high              2017-05-03
low               2016-11-09
volume            2017-03-10
EWMA26            2018-02-05
EWMA12            2018-02-05
EWMA9             2018-02-05
MACD              2018-02-05
Returns           2018-02-06
log_return        2018-02-06
Bollinger Upper   2018-02-05
Bollinger Lower   2017-07-14
Heiking_Close     2017-06-14
Heiking High      2017-05-03
Heiking Low       2016-11-09
Heiking Open      2017-12-21
log_MACD          2018-02-05
Variance12        2017-12-11
dtype: datetime64[ns]


,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,497.00,497.00,497.00,497.00,497.00,497.00,497.00,497.00,149.00,496.00,496.00,496.00,496.00,497.00,497.00,497.00,496.00,497.00,496.00
mean,-0.00,"3,042,093.59",0.04,-0.05,34.90,-0.05,-0.02,-0.02,-inf,0.01,0.01,0.13,-0.21,-0.00,0.04,-0.05,-0.01,-0.03,"328,148.34"
std,0.06,"3,020,119.25",0.05,0.06,29.00,0.13,0.08,0.07,nan,0.06,0.06,0.10,0.16,0.03,0.05,0.06,0.06,0.06,"464,647.70"
min,-0.34,"463,059.45",0.00,-0.58,0.00,-0.49,-0.33,-0.28,-inf,-0.25,-0.29,0.00,-0.99,-0.15,0.00,-0.58,-0.25,-0.19,"3,706.92"
25%,-0.03,"740,000.00",0.01,-0.07,15.81,-0.13,-0.07,-0.05,-5.50,-0.02,-0.02,0.06,-0.27,-0.02,0.01,-0.07,-0.03,-0.07,"43,783.90"
50%,-0.00,"1,797,000.00",0.02,-0.03,29.20,-0.05,-0.02,-0.02,-4.29,0.00,0.00,0.10,-0.18,-0.00,0.02,-0.03,-0.01,-0.02,"152,863.73"
75%,0.02,"4,601,423.13",0.05,-0.02,43.27,0.01,0.01,0.01,-3.25,0.03,0.03,0.17,-0.10,0.01,0.05,-0.02,0.02,0.00,"367,989.29"
max,0.26,"13,794,997.00",0.34,0.00,233.56,0.55,0.38,0.32,-1.29,0.37,0.32,0.76,0.01,0.14,0.34,0.00,0.29,0.17,"2,267,863.23"


describe_dataset


...

,open,close,high,low,volume,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,208.00,208.00,208.00,208.00,208.00,208.00,208.00,208.00,95.00,207.00,207.00,207.00,204.00,208.00,208.00,208.00,207.00,208.00,207.00
mean,-0.02,"1,994.46",0.07,-0.09,3.08,-0.03,-0.01,-0.00,-inf,0.03,0.00,0.27,-0.51,0.00,0.07,-0.09,-0.01,-0.03,360.57
std,0.19,"1,105.04",0.16,0.50,5.00,0.20,0.17,0.15,nan,0.25,0.22,0.21,0.66,0.10,0.16,0.50,0.21,0.06,348.26
min,-1.36,654.01,0.00,-7.05,0.00,-0.97,-0.75,-0.67,-inf,-0.57,-0.84,-0.03,-4.44,-0.57,0.00,-7.05,-1.01,-0.25,0.79
25%,-0.01,"1,047.88",0.00,-0.05,0.10,-0.11,-0.08,-0.08,-4.42,-0.05,-0.05,0.13,-0.53,-0.00,0.00,-0.05,-0.07,-0.05,126.15
50%,0.00,"1,447.00",0.00,0.00,0.93,-0.01,-0.00,0.00,-3.96,0.00,0.00,0.21,-0.31,0.00,0.00,0.00,0.00,-0.00,270.47
75%,0.02,"2,858.75",0.06,0.00,3.98,0.07,0.07,0.07,-3.35,0.06,0.06,0.36,-0.17,0.02,0.06,0.00,0.05,0.02,438.58
max,0.57,"4,997.00",1.03,0.00,37.77,0.57,0.52,0.48,-2.42,1.75,1.01,1.08,-0.00,0.49,1.03,0.00,0.84,0.06,"1,555.51"


describe_dataset


low               2017-05-25
volume            2013-08-18
amount            2013-10-20
avg_price         2013-11-27
open              2013-11-18
close             2013-07-03
high              2013-06-13
quantity          2013-08-18
EWMA26            2013-11-18
EWMA12            2013-11-18
EWMA9             2013-11-18
MACD              2015-06-04
Returns           2013-09-06
log_return        2013-09-06
Bollinger Upper   2016-05-28
Bollinger Lower   2013-11-18
Heiking_Close     2018-02-16
Heiking High      2013-06-13
Heiking Low       2017-05-25
Heiking Open      2013-11-18
log_MACD          2013-11-29
Variance12        2013-06-13
dtype: datetime64[ns]


low               2013-06-15
volume            2017-12-07
amount            2015-01-14
avg_price         2013-09-06
open              2013-09-06
close             2017-12-18
high              2013-10-19
quantity          2013-07-05
EWMA26            2018-02-05
EWMA12            2013-12-18
EWMA9             2013-12-07
MACD              2018-02-05
Returns           2013-11-18
log_return        2013-11-18
Bollinger Upper   2013-12-18
Bollinger Lower   2014-08-14
Heiking_Close     2013-12-07
Heiking High      2013-10-19
Heiking Low       2013-06-15
Heiking Open      2013-12-07
log_MACD          2018-02-06
Variance12        2017-12-18
dtype: datetime64[ns]


,low,volume,amount,avg_price,open,close,high,quantity,EWMA26,EWMA12,EWMA9,MACD,Returns,log_return,Bollinger Upper,Bollinger Lower,Heiking_Close,Heiking High,Heiking Low,Heiking Open,log_MACD,Variance12
count,"1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00","1,681.00",684.00,"1,680.00","1,680.00","1,680.00","1,680.00","1,681.00","1,681.00","1,681.00","1,680.00","1,681.00","1,680.00"
mean,-inf,"2,147,448.68",-1.67,-0.00,-0.00,"5,437.67",0.03,-3.06,-0.03,-0.01,-0.01,-inf,0.00,0.00,0.11,-0.17,-0.00,0.03,-inf,-0.00,-0.01,544.91
std,nan,"7,653,198.97",0.65,0.02,0.05,"10,676.18",0.04,1.00,0.12,0.08,0.06,nan,0.05,0.05,0.10,0.19,0.02,0.04,nan,0.05,0.06,"1,411.80"
min,-inf,24.55,-4.93,-0.13,-0.35,190.01,0.00,-7.82,-0.76,-0.51,-0.43,-inf,-0.24,-0.28,-0.00,-1.84,-0.28,0.00,-inf,-0.41,-0.29,2.83
25%,-0.04,"59,930.05",-2.10,-0.01,-0.02,960.00,0.01,-3.69,-0.07,-0.04,-0.03,-4.75,-0.01,-0.01,0.04,-0.21,-0.01,0.01,-0.04,-0.02,-0.03,35.22
50%,-0.02,"131,661.52",-1.56,-0.00,-0.00,"1,690.11",0.02,-2.97,-0.01,-0.01,-0.00,-3.80,0.00,0.00,0.08,-0.10,-0.00,0.02,-0.02,-0.00,-0.01,70.72
75%,-0.01,"487,363.80",-1.19,0.01,0.01,"2,960.00",0.03,-2.38,0.03,0.02,0.02,-3.12,0.02,0.02,0.14,-0.06,0.01,0.03,-0.01,0.01,0.02,208.76
max,0.00,"108,023,418.47",0.17,0.21,0.26,"69,099.00",0.57,0.41,0.50,0.40,0.38,-1.42,0.41,0.35,0.80,0.01,0.18,0.57,0.00,0.37,0.17,"10,544.25"


### Save Data for Datasets

Save the data in objects to be consumed later. 

In [22]:

def drop_the_first_row(dataset):
    dataset.df.drop(dataset.df.index[:1], inplace=True)
    return dataset

all_datasets(drop_the_first_row,rescaled_datasets)    
all_datasets(drop_the_first_row,datasets)    
all_datasets(drop_the_first_row,log_datasets)    
all_datasets(drop_the_first_row,log_divided_close_datasets) 

def save_datasets(datasets,filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(datasets, output, pickle.HIGHEST_PROTOCOL)
    
save_datasets(rescaled_datasets, 'datasets/rescaled_datasets.pkl')
save_datasets(datasets, 'datasets/datasets.pkl')
save_datasets(log_datasets, 'datasets/log_datasets.pkl')
save_datasets(log_divided_close_datasets, 'datasets/log_divided_close_datasets.pkl')




drop_the_first_row


drop_the_first_row


drop_the_first_row


drop_the_first_row


drop_the_first_row


...

drop_the_first_row


drop_the_first_row


drop_the_first_row


drop_the_first_row


drop_the_first_row


NameError: name 'score_clean_close' is not defined